## Spark Streaming Twitter

In [ ]:
# se não tiver instalado, instale os pacotes a baixo
#!pip install requests_oauthlib
#!pip install twython
#!pip install nltk

In [ ]:
# modulos que serão usados
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from requests_oauthlib import OAuth2Session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import requests
import time
import string
import ast
import json

In [ ]:
# importando pacotes NLTK
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [2]:
import twitter

In [ ]:
# variaval que define o intervalo que vamos obter os twits 
INTERNALO_BATCH = 5

In [ ]:
# criando o streamingContext
ssc = StreamingContext(sc, INTERNALO_BATCH)

### Treinando classificador de análise de sentimento

Uma parte essencial da criação de um algoritmo de análise de sentimento (ou qualquer outro algoritmo de mineração de dados) é ter um conjunto de dados abrangente "Corpus" para o aprendizado, bem como um conjunto de dado de teste para garantir que a precisão do seu algoritmo atenda aos padrões que você espera. Isso também permitirá que voê ajuste o seu algoritmo a fim de deduzir melhores (ou mais precisas) caracteristicas da linguagem natural que voce poderia extrair do texto, e que vão contribuir para análise de sentimento em vez de usar uma abordagem genérica. tomaremos como base o dataset de treino fornecido pela universidade de Michigam para competições do kaggle

label
### 1 para sentimento positivo
### 0 para sentimento negativo

In [ ]:
arquivo = sc.textFile('2-Arquivos-Cap10/dataset_analise_sentimento.csv')

In [ ]:
type(arquivo)

In [ ]:
# removendo o cabeçalho
header = arquivo.take(1)[0]
dataset = arquivo.filter(lambda x: x != header)

In [ ]:
type(dataset)

In [ ]:
# função para separar colunas em cada linha, cria uma tupla e remove pontuação
def get_row(linha):
    row = linha.split(',') # separo por virgula
    sentimento = row[1] # pego a linha de indice 1 que é a posição do sentimento 
    tweet = row[3].strip() # dividindo cada uma das linhas com o strip
    translator = str.maketrans({key: None for key in string.punctuation}) # removendo a pontuação com o string.punctuation
    tweet = tweet.translate(translator)
    tweet = tweet.split(" ") # separa por espaço
    tweet_lower = [] # adicionando palavras em uma nova lista porem em lower case
    for word in tweet:
        tweet_lower.append(word.lower())
    return (tweet_lower,sentimento)

In [ ]:
# aplica a função a cada linha do dataset
dataset_treino = dataset.map(lambda x: get_row(x))

In [ ]:
# cria um objeto SentimentAnalyzer
sentiment_analyzer = SentimentAnalyzer()

In [ ]:
nltk.download('stopwords')

In [ ]:
from IPython.display import Image
Image(url='2-Arquivos-Cap10/ntlkdata.png')

In [ ]:
# obtem a lista de stopwords em ingles
stopword_all = []
for word in stopwords.words('english'):
    stopword_all.append(word)
    stopword_all.append(word+'_NEG')

In [ ]:
# obtem 10.000 tweets do dataset de treino 
dataset_treino_amostra = dataset_treino.take(10000)

In [ ]:
dataset_treino_amostra

In [ ]:
all_words_neg = sentiment_analyzer.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopword_all]

In [ ]:
# cria um unigram e extrai as features
unigram_feats = sentiment_analyzer.unigram_word_feats(all_words_neg_nostops, top_n=200)
sentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = sentiment_analyzer.apply_features(dataset_treino_amostra)

In [ ]:
type(training_set)

In [ ]:
print(training_set)

In [ ]:
# treinando modelo
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyzer.train(trainer, training_set)

In [ ]:
test_sentence1 = [(['this','program','is','bad'],'')]
test_sentence2 = [(['thogh','day', 'at','work','today'],'')]
test_sentence3 = [(['good','wonderful','amazing','awsomw'],'')]
t1 = sentiment_analyzer.apply_features(test_sentence1)
t2 = sentiment_analyzer.apply_features(test_sentence2)
t3 = sentiment_analyzer.apply_features(test_sentence3)


In [4]:
consumer_key = "XXX"
consumer_secret = "XXX"
access_token = "XXX"
access_secret = "XXX"


In [1]:
!pip install python-twitter


In [5]:
t = twitter.Api(
    consumer_key = consumer_key, 
    consumer_secret = consumer_secret, 
    access_token_key = access_token, 
    access_token_secret = access_secret, 
    tweet_mode = 'extended')

t.

In [ ]:
# criando objeto de autenticação do twitter
auth = requests_oauthlib.OAuth1(consumer_key, consumer_secret, access_token, access_secret)

In [ ]:
# configurando stream
rdd = ssc.sparkContext.parallelize([0])
stream = ssc.queueStream([],default=rdd)

In [ ]:
type(stream)

In [ ]:
# numero de tweets por update
NUM_TWEETS = 50

In [ ]:

            
# Essa função conecta ao Twitter e retorna um número específico de Tweets (NUM_TWEETS)
def tfunc(t, rdd):
  return rdd.flatMap(lambda x: stream_twitter_data())

def stream_twitter_data():
  response = requests.get(filter_url, auth = auth, stream = True)
  print(filter_url, response)
  count = 0
  for line in response.iter_lines():
    try:
      if count > NUM_TWEETS:
        break
      post = json.loads(line.decode('utf-8'))
      contents = [post['text']]
      count += 1
      yield str(contents)
    except:
      result = False

In [ ]:
stream = stream.transform(tfunc)

In [ ]:
coord_stream = stream.map(lambda x: ast.literal_eval(x))

In [ ]:
def classifica_tweet(tweet):
    sentence = [(tweet), '']
    test_set = sentiment_analyzer.apply_features(sentence)
    print(tweet, classifier.classify(test_set[0][0]))
    return (tweet, classifier.classify(test_set[0][0]))

In [ ]:
# retorna o txt odo twitter
def get_tweet_text(rdd):
    for line in rdd:
        tweet = line.strip()
        print(tweet)
        translator = str.maketrans({key: None for key in string.punctuation}) # removendo a pontuação com o string.punctuation
        tweet = tweet.translate(translator)
        tweet = tweet.split(' ') # separa por espaço
        tweet_lower = [] # adicionando palavras em uma nova lista porem em lower case
        for word in tweet:
            tweet_lower.append(word.lower())
        return (classifica_tweet(tweet_lower))

In [ ]:
resultados = []

In [ ]:
def output_rdd(rdd):
    global resultados
    pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
    counts = pairs.reduceByKey(add)
    output = []
    for count in counts.collect():
        output.append(count)
    result = (time.strftime("%I:%M:%S"), output)
    resultados.append(result)
    print(result)

In [ ]:
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

In [ ]:
ssc.start()

In [ ]:
count = True
while count:
    if(len(resultados) > 5):
        count = False

In [ ]:
rdd_save = '2-Arquivos-Cap10/r'+time.strftime('%I%M%S')
resultado_rdd = sc.parallelize(resultado)
resultado_rdd.saveAsTextFilee(rdd_save)

In [ ]:
resultado_rdd.collect()

In [ ]:
ssc.stop()